In [2]:
import torch
import numpy as np
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from tqdm.notebook import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [3]:
import torchvision.transforms as transforms

In [4]:
TRAIN_DATA_PATH = "train"
TEST_DATA_PATH = "test"
training_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform = ToTensor())
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform = ToTensor())


In [5]:
batch_size = 256

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [6]:
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([256, 3, 48, 48])
Shape of y: torch.Size([256]) torch.int64


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=64,            
                kernel_size=3,              
                stride=1,                   
                padding=1,                  
            ),                              
            nn.ReLU(),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(64, 128, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.conv3 = nn.Sequential(         
            nn.Conv2d(128, 256, 3, 1, 1),     
            nn.ReLU(),                                      
        )
        self.conv4 = nn.Sequential(         
            nn.Conv2d(256, 256, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # for 6th layer
        self.conv5 = nn.Sequential(         
            nn.Conv2d(256, 128, 3, 1, 1),     
            nn.ReLU(), 
        )

        self.conv6 = nn.Sequential(         
            nn.Conv2d(128, 64, 3, 1, 1),     
            nn.ReLU(),
            nn.MaxPool2d(2), 
        )

        # for 7th layer
        self.conv7 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, 1),     
            nn.ReLU(), 
            
        )
        self.dropout = nn.Dropout(0)
        # fully connected layer


        self.out = nn.Sequential(
            # change accordingly to number of layers
            nn.Linear(6*6*32,7),
            nn.Softmax()
        )

        self.transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((48, 48)),
        transforms.Normalize((0.5,), (0.5,))
        ])

    def forward(self, x):
        x = self.transform(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.dropout(x)
        # flatten 
        x = x.view(x.size(0), -1)      
        output = self.out(x)
        return output

model = CNN().to(device)
print(model)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv5): Sequential(
    (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv6): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv7): Sequential(
    (

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    pbar = tqdm(total=28709//256+1)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        pbar.update(1)
    pbar.close()


In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct*100

In [97]:
epochs = 64
best_acc = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(train_dataloader, model, loss_fn, optimizer)
    acc = test(test_dataloader, model, loss_fn)

    if acc > best_acc:
        # save the state dictionary of the model's weights and other parameters to a file
        torch.save(model.state_dict(), 'model cnn 7 new.pth')
        print('saved')
        best_acc = acc
print("Done!")

Epoch 1
-------------------------------


  0%|          | 0/113 [00:00<?, ?it/s]

d:\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 1.860126  [    0/28709]
loss: 1.807597  [25600/28709]
Test Error: 
 Accuracy: 33.5%, Avg loss: 1.826123 

saved
Epoch 2
-------------------------------


  0%|          | 0/113 [00:00<?, ?it/s]

...

loss: 1.590279  [    0/28709]
loss: 1.647600  [25600/28709]
Test Error: 
 Accuracy: 43.8%, Avg loss: 1.737488 

Epoch 63
-------------------------------


  0%|          | 0/113 [00:00<?, ?it/s]

loss: 1.545720  [    0/28709]
loss: 1.639486  [25600/28709]
Test Error: 
 Accuracy: 44.0%, Avg loss: 1.735996 

saved
Epoch 64
-------------------------------


  0%|          | 0/113 [00:00<?, ?it/s]

loss: 1.601497  [    0/28709]
loss: 1.560881  [25600/28709]
Test Error: 
 Accuracy: 43.7%, Avg loss: 1.739782 

Done!


In [9]:
# torch.save(model.state_dict(), "model 7 layer 55 acc.pth")
# print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


In [13]:
model = CNN()
model.load_state_dict(torch.load("model cnn 7 new.pth"))
model.to(device)



CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv5): Sequential(
    (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv6): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv7): Sequential(
    (

In [15]:
print('cnn')
test(test_dataloader, model, loss_fn)

cnn
Test Error: 
 Accuracy: 44.0%, Avg loss: 1.735996 



43.99554193368627